# Part 3: Estimating inundation extent using HAND methodology


In this example we will use some of GRASS GIS hydrology tools, namely:

* [r.watershed](https://grass.osgeo.org/grass-stable/manuals/r.watershed.html): for computing flow accumulation, drainage direction, the location of streams and watershed basins; it does not need sink filling because of using the least-cost-path to route flow out of sinks
* [r.lake](https://grass.osgeo.org/grass-stable/manuals/r.lake.html): to fill a lake to a target water level from a given start point or seed raster
* [r.lake.series](https://grass.osgeo.org/grass-stable/manuals/r.lake.series.html): addon which runs r.lake for different water levels
* [r.stream.distance](https://grass.osgeo.org/grass-stable/manuals/r.stream.distance.html): for computing the distance to streams or outlet, the relative elevation above streams; the distance and the elevation are calculated along watercourses

First, let's create a new mapset *flooding* in nc_spm_08_grass7 sample dataset:

In [ ]:
%%bash
grass -c -e ~/grassdata/nc_spm_08_grass7/flooding

Initialize GRASS session:

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "nc_spm_08_grass7", "flooding")

Modules r.stream.distance and r.lake.series are addons and we need to install them first:

In [ ]:
gs.run_command("g.extension", extension="r.stream.distance")
gs.run_command("g.extension", extension="r.lake.series")

## Compute HAND raster

We will estimate inundation extent using the Height Above Nearest Drainage methodology ([A.D. Nobre, 2011](https://doi.org/10.1016/j.jhydrol.2011.03.051)). We will compute the HAND terrain model representing the differences in elevation between each grid cell and the elevations of the flowpath-connected downslope grid cells where the flow enters the channel.

First we compute the flow accumulation, drainage and streams (with a threshold value of 100000). We convert the streams to vector for better visualization.

In [ ]:
gs.run_command("g.region", raster="elevation")
gs.run_command("r.watershed", elevation="elevation", accumulation="flowacc",
               drainage="drainage", stream="rwatershed_streams", threshold=100000)
gs.run_command("r.to.vect", input="rwatershed_streams", output="rwatershed_streams", type="line")

fllowacc_map = gj.Map()
fllowacc_map.d_rast(map="flowacc")
fllowacc_map.d_vect(map="rwatershed_streams", width=2, color="blue")
fllowacc_map.d_legend(raster="flowacc", range="0,1000")
fllowacc_map.show()

Let's zoom in to see the flow accumulation raster better:

In [ ]:
flowacc_map = gj.InteractiveMap()
flowacc_map.add_raster("flowacc")
flowacc_map.add_vector("rwatershed_streams")
flowacc_map.show()

Now we use r.stream.distance with output parameter difference to compute a new raster map where each cell is the elevation difference between the cell and the the cell on the stream where the cell drains.

In [ ]:
gs.run_command("r.stream.distance", stream_rast="rwatershed_streams", direction="drainage",
               elevation="elevation", method="downstream", difference="above_stream")
gs.run_command("r.colors", map="above_stream", color="elevation")

Compare original elevation and HAND raster:

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="elevation", color_map="elevation",
             position=(0.5, 1), height=3000, perspective=12, zexag=5)
map3d.overlay.d_legend(raster="elevation", at=(5, 80, 87, 92), flags="b", border_color="none")
map3d.show()

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="above_stream", color_map="above_stream",
             position=(0.5, 1), height=3000, perspective=12, zexag=5)
map3d.overlay.d_legend(raster="above_stream", at=(5, 80, 90, 95), flags="b", border_color="none")
map3d.show()

## Inundation
Before we compute the inundation, we will look at how r.lake works. We compute a lake from a specified coordinate pair and water level:

In [ ]:
gs.run_command("r.lake", elevation="elevation", water_level=90, lake="lake", coordinates=[637877, 218475])

lake_map = gj.Map()
lake_map.d_rast(map="elevation")
lake_map.d_rast(map="lake")
lake_map.d_legend(raster="lake", label_values="0.1,5,10,15", digits=0, at=(1, 40, 90, 95))
lake_map.show()

Now instead of the elevation raster we use the HAND raster to simulate 5-meter inundation and, as the seed we specify the entire stream.

In [ ]:
gs.run_command("r.lake", elevation="above_stream", water_level=5, lake="flood", seed="rwatershed_streams")

hand_map = gj.Map()
hand_map.d_rast(map="above_stream")
hand_map.d_rast(map="flood")
hand_map.d_legend(raster="flood", label_values="0.1,4,8", digits=0, at=(1, 40, 90, 95))
hand_map.show()

With the r.lake.series addon we can create a series of inundation maps with rising water levels:

In [ ]:
from io import StringIO
import pandas as pd

gs.run_command("r.lake.series", elevation="above_stream", start_water_level=0, end_water_level=5, 
               water_level_step=0.5, output="inundation", seed_raster="rwatershed_streams", quiet=True)
gs.run_command("t.rast.colors", input="inundation", color="water")
pd.read_csv(StringIO(gs.read_command("t.rast.list", input="inundation", separator="comma")))

r.lake.series creates a space-time dataset. We can use the [temporal modules](https://grass.osgeo.org/grass-stable/manuals/temporal.html) to further work with the data. For example, we could further compute the volume and extent of flood water using t.rast.univar:

In [ ]:
pd.read_csv(StringIO(gs.read_command("t.rast.univar", input="inundation", separator="comma")))

Let's visualize the results:

In [ ]:
timemap = gj.TimeSeriesMap()
timemap.d_rast(map="elevation_shade")
timemap.d_vect(map="streets_wake", color="#4D4D4D")
timemap.add_raster_series("inundation")
timemap.d_legend(at=(1, 40, 90, 95))
timemap.show()